<a href="https://colab.research.google.com/github/saks0106/DL_Frequent_Lookout/blob/main/Markov_Model_Binary_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2023-10-09 16:29:49--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt 100%[===================>]  26.00K  --.-KB/s    in 0.001s  

2023-10-09 16:29:50 (17.3 MB/s) - ‘edgar_allan_poe.txt’ saved [26622/26622]

--2023-10-09 16:29:50--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP re

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [ ]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [ ]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [ ]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [ ]:
# collect data into lists
#We have poem|author,we want to find author|peom using bayes rule
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [ ]:
input_texts[0], labels[0]

('lo death hath reard himself a throne', 0)

In [ ]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [ ]:
train_text[0], Ytrain[0]

('this is the pit from which we starks were digged', 1)

In [ ]:
idx = 1
word2idx = {'<unk>': 0}
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

len(word2idx)
#'hall': 989,'touch': 990,'knows': 991,'forth': 992,'lets': 993,'curl': 994

2540

In [ ]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [ ]:
train_text_int[100:105]

[[28, 26, 384, 70, 385, 386, 138, 387],
 [226, 388, 263, 389, 365, 142, 7, 390, 388],
 [391, 392, 126, 23, 393, 121, 394, 3, 395],
 [26, 72, 396, 397, 3, 398, 9, 399],
 [194, 400, 13, 401, 5, 128, 108, 110]]

In [ ]:
np.ones((100, 100))

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [ ]:
# initialize A and pi matrices - for both classes
V = len(word2idx) #in practice we use M

A0 = np.ones((V, V))
pi0 = np.ones(V) #starting words 1D list

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [ ]:
A0[1].shape,A0[2] , A1[2]

((2540,),
 array([0.00039078, 0.00039078, 0.00039078, ..., 0.00039078, 0.00039078,
        0.00039078]),
 array([0.00038745, 0.00077489, 0.00077489, ..., 0.00038745, 0.00038745,
        0.00038745]))

In [ ]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None #For tracking pi
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0) #['this is the pit from which we starks were digged', 1],[],[]
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [ ]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [ ]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [ ]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.3306501547987616, 0.6693498452012384)

In [ ]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]

      # update last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [ ]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [ ]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9944272445820433


In [ ]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8311688311688312


In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 525,    9],
       [   0, 1081]])

In [ ]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[101,  83],
       [  8, 347]])

In [ ]:
f1_score(Ytrain, Ptrain)

0.9958544449562414

In [ ]:
f1_score(Ytest, Ptest)

0.884076433121019